- Membuat ETL Pipeline menggunakan Luigi
- Proposed ETL Pipeline design adalah sebagai berikut

<center>
<img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-intro-to-data-eng/15-01.png" width=70%>
</center>

- Task yang harus dilakukan adalah:
    1. Membuat Pipeline Extract dari source:
        - [API Payment Data](https://shandytepe.github.io/payment.json)
        - Database Hotel
    
       Setelah berhasil melakukan Extract, simpan output `.csv` ke dalam folder `live_class_w8/data/raw`
    
    2. Membuat Pipeline Validation dari Pipeline Extract data. Proses validasi yang dilakukan adalah:
        - Check Data Shape
        - Check Data Types
        - Check Missing Values

    3. Membuat Pipeline Transform dari Pipeline Extract Data. Proses yang dilakukan adalah:
        - Join Data dari beberapa source, menjadi satu table data yang baru
            - Source yang digunakan untuk membuat table baru adalah:
                - `payment` (API Payment Data)
                - `reservation` (DB Hotel)
                - `customer` (DB Hotel)


<center>
<img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-intro-to-data-eng/15-02.png" width=80%>
</center>


- Cleaning data

    Setelah berhasil melakukan proses Transform, simpan output `.csv` ke dalam folder `live_class_w8/data/transform`

    4. Membuat Pipeline Load dari Pipeline Transform. Memasukkan table yang baru ke dalam Data Warehouse yang sudah dibuat. Untuk membuat Data Warehouse, bisa melihat dokumentasi di https://github.com/shandytepe/live-class-w8-intro-to-data-eng/tree/main/docker-db

    5. Menggabungkan semua Pipeline menjadi satu dengan menggunakan Luigi

    6. Set schedule pada Pipelien yang sudah dibuat dengan menggunakan `cron`. Set schedule pipeline untuk dijalankan tiap 2 menit sekali. Sehingga, syntax `cron` nya adalah sebagai berikut

        ```bash
        */2 * * * * /path/to/pipeline.sh
        ```

    7. Melakukan proses `UPSERT` untuk mengecek robustness dari pipeline yang dibuat


# Extract

## Data Hotel

In [4]:
import requests
import json
import pandas as pd

# URL of the JSON file
url = "https://shandytepe.github.io/payment.json"

# Fetch the JSON file from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content
    data = response.json()  # Automatically parses JSON into a Python dictionary
    print(data)  # Print the JSON content
else:
    print(f"Failed to fetch JSON. HTTP Status Code: {response.status_code}")


{'payment_data': [{'payment_id': 1, 'reservation_id': 1, 'provider': 'Ovo', 'account_number': 38137149, 'payment_status': 'Success', 'payment_date': '2020-10-20 17:07:04.406780', 'expire_date': '2020-10-28 18:10:37.175108'}, {'payment_id': 2, 'reservation_id': 2, 'provider': 'BCA', 'account_number': 42103729, 'payment_status': 'Success', 'payment_date': '2015-02-28 16:44:56.703511', 'expire_date': '2015-03-08 05:27:33.443131'}, {'payment_id': 3, 'reservation_id': 3, 'provider': 'Permata', 'account_number': 58689635, 'payment_status': 'Success', 'payment_date': '2022-04-03 13:27:30.811447', 'expire_date': '2022-04-06 14:20:57.846492'}, {'payment_id': 4, 'reservation_id': 4, 'provider': 'BNI', 'account_number': 107161965, 'payment_status': 'Success', 'payment_date': '2019-10-04 08:39:56.034805', 'expire_date': '2019-10-08 02:01:21.209415'}, {'payment_id': 5, 'reservation_id': 5, 'provider': 'BSI', 'account_number': 14334131, 'payment_status': 'Failed', 'payment_date': None, 'expire_date'

In [5]:
raw_data = data['payment_data']


In [35]:
df = pd.DataFrame(raw_data)
df

,payment_id,reservation_id,provider,account_number,payment_status,payment_date,expire_date
0,1,1,Ovo,38137149,Success,2020-10-20 17:07:04.406780,2020-10-28 18:10:37.175108
1,2,2,BCA,42103729,Success,2015-02-28 16:44:56.703511,2015-03-08 05:27:33.443131
2,3,3,Permata,58689635,Success,2022-04-03 13:27:30.811447,2022-04-06 14:20:57.846492
3,4,4,BNI,107161965,Success,2019-10-04 08:39:56.034805,2019-10-08 02:01:21.209415
4,5,5,BSI,14334131,Failed,None,2020-06-14 12:02:55.669148
...,...,...,...,...,...,...,...
396,397,397,Permata,87836019,Success,2018-08-05 09:22:05.983116,2018-08-27 12:44:56.532991
397,398,398,Mandiri,50997930,Failed,None,2022-04-19 12:35:51.127022
398,399,399,Ovo,44305033,Success,2015-12-12 22:35:15.569310,2015-12-19 19:28:08.809478
399,400,400,BNI,84875963,Success,2021-07-29 16:38:06.052542,2021-08-02 22:27:28.713637


## Make Connection Database

In [16]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from typing import Optional

def postgres_engine() -> Optional[create_engine]:
    """
    Create and return a SQLAlchemy engine for a PostgreSQL database.

    Returns:
        Optional[create_engine]: A SQLAlchemy engine object, or None if an error occurs.
    """
    # Load environment variables from .env file
    load_dotenv("Week 8/.env")

    # Fetch environment variables
    db_user = os.getenv('DB_USER')
    db_password = os.getenv('DB_PASSWORD')
    db_host = os.getenv('DB_HOST')
    db_port = os.getenv('DB_PORT')
    db_name = os.getenv('DB_NAME')

    # Debug: Print the retrieved values
    print(f"DB_USER: {db_user}")
    print(f"DB_PASSWORD: {db_password}")
    print(f"DB_HOST: {db_host}")
    print(f"DB_PORT: {db_port}")
    print(f"DB_NAME: {db_name}")

    # Check if all required environment variables are set
    if not all([db_user, db_password, db_host, db_port, db_name]):
        raise ValueError("One or more required environment variables are not set.")

    # Create the database URL
    database_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

    # Create and return the engine
    engine = create_engine(database_url)
    return engine

In [17]:
try:
    engine = postgres_engine()
    print("Successfully connected to the database")
    # Perform database operations
except Exception as e:
    print(f"An error occurred: {e}")

DB_USER: postgres
DB_PASSWORD: cobapassword
DB_HOST: localhost
DB_PORT: 5434
DB_NAME: hotel_analysis_db
Successfully connected to the database


## Retrieve Table from DB

In [25]:
def read_sql(table_name: str) -> pd.DataFrame:
    # Query the database
    query = f"SELECT * FROM public.{table_name};"
    df = pd.read_sql(query, engine)

    # Display the results
    return df  # Return the DataFrame



In [26]:
# Call the function and store the result
df_reservation = read_sql('reservation')
df_customer = read_sql('customer')

df_customer.head()
df_reservation.head()

,reservation_id,customer_id,reservation_date,start_date,end_date,total_price,review,rating,created_at
0,1,885,2020-10-03 14:38:18.907,2020-10-28 18:10:37.175,2020-11-07 06:56:19.627,7000000.0,hated hotel arrested talking loud arrived hote...,2,2025-01-26 06:11:04.602576
1,2,144,2015-02-17 20:44:11.696,2015-03-08 05:27:33.443,2015-03-13 01:01:28.449,15000000.0,great location 110 priceline hotel block tram ...,3,2025-01-26 06:11:04.602576
2,3,560,2022-03-19 12:18:48.993,2022-04-06 14:20:57.846,2022-04-10 08:06:20.564,3000000.0,"really great, great stay, rooms large clean go...",5,2025-01-26 06:11:04.602576
3,4,710,2019-10-01 12:33:58.891,2019-10-08 02:01:21.209,2019-10-19 06:25:22.948,15000000.0,great time wife recently opportunity stay sher...,2,2025-01-26 06:11:04.602576
4,5,606,2020-05-28 23:25:59.539,2020-06-14 12:02:55.669,2020-06-22 20:59:13.732,5000000.0,beautiful just returned night stay gran melia ...,2,2025-01-26 06:11:04.602576


In [28]:
df_reservation.head()
df_customer.head()

,customer_id,first_name,last_name,email,phone,created_at
0,1,Surya,Nainggolan,suryanainggolan@hotmail.com,(0904) 694-4591,2024-02-09 13:59:44.733151
1,2,Estiawan,Saragih,estiawansaragih@hotmail.com,+62-62-782-9299,2024-02-09 13:59:44.733151
2,3,Mahesa,Wastuti,mahesawastuti@yahoo.com,+62 (46) 932-1963,2024-02-09 13:59:44.733151
3,4,Drajat,Nainggolan,drajatnainggolan@yahoo.com,+62-042-726-1498,2024-02-09 13:59:44.733151
4,5,Simon,Wacana,simonwacana@hotmail.com,(0007) 948 5515,2024-02-09 13:59:44.733151


# Transform

In [36]:
df

,payment_id,reservation_id,provider,account_number,payment_status,payment_date,expire_date
0,1,1,Ovo,38137149,Success,2020-10-20 17:07:04.406780,2020-10-28 18:10:37.175108
1,2,2,BCA,42103729,Success,2015-02-28 16:44:56.703511,2015-03-08 05:27:33.443131
2,3,3,Permata,58689635,Success,2022-04-03 13:27:30.811447,2022-04-06 14:20:57.846492
3,4,4,BNI,107161965,Success,2019-10-04 08:39:56.034805,2019-10-08 02:01:21.209415
4,5,5,BSI,14334131,Failed,None,2020-06-14 12:02:55.669148
...,...,...,...,...,...,...,...
396,397,397,Permata,87836019,Success,2018-08-05 09:22:05.983116,2018-08-27 12:44:56.532991
397,398,398,Mandiri,50997930,Failed,None,2022-04-19 12:35:51.127022
398,399,399,Ovo,44305033,Success,2015-12-12 22:35:15.569310,2015-12-19 19:28:08.809478
399,400,400,BNI,84875963,Success,2021-07-29 16:38:06.052542,2021-08-02 22:27:28.713637


In [30]:
df_reservation

,reservation_id,customer_id,reservation_date,start_date,end_date,total_price,review,rating,created_at
0,1,885,2020-10-03 14:38:18.907,2020-10-28 18:10:37.175,2020-11-07 06:56:19.627,7000000.0,hated hotel arrested talking loud arrived hote...,2,2025-01-26 06:11:04.602576
1,2,144,2015-02-17 20:44:11.696,2015-03-08 05:27:33.443,2015-03-13 01:01:28.449,15000000.0,great location 110 priceline hotel block tram ...,3,2025-01-26 06:11:04.602576
2,3,560,2022-03-19 12:18:48.993,2022-04-06 14:20:57.846,2022-04-10 08:06:20.564,3000000.0,"really great, great stay, rooms large clean go...",5,2025-01-26 06:11:04.602576
3,4,710,2019-10-01 12:33:58.891,2019-10-08 02:01:21.209,2019-10-19 06:25:22.948,15000000.0,great time wife recently opportunity stay sher...,2,2025-01-26 06:11:04.602576
4,5,606,2020-05-28 23:25:59.539,2020-06-14 12:02:55.669,2020-06-22 20:59:13.732,5000000.0,beautiful just returned night stay gran melia ...,2,2025-01-26 06:11:04.602576
...,...,...,...,...,...,...,...,...,...
395,396,552,2021-03-16 16:46:20.854,2021-03-18 08:22:51.726,2021-03-21 08:03:21.100,15000000.0,perfect families booked hotel based rave revie...,2,2025-01-26 06:11:04.696962
396,397,617,2018-08-01 23:40:55.197,2018-08-27 12:44:56.532,2018-09-01 19:05:38.807,5000000.0,hotel fab booked hotel reading reviews website...,3,2025-01-26 06:11:04.696962
397,398,258,2022-03-29 04:38:14.964,2022-04-19 12:35:51.127,2022-04-21 03:06:16.519,15000000.0,solid business hotel near embassy stayed hotel...,1,2025-01-26 06:11:04.696962
398,399,304,2015-12-01 20:42:11.606,2015-12-19 19:28:08.809,2015-12-22 02:52:27.249,1000000.0,nice hotel husband stayed crown plaza spring b...,5,2025-01-26 06:11:04.696962


In [31]:
df_customer

,customer_id,first_name,last_name,email,phone,created_at
0,1,Surya,Nainggolan,suryanainggolan@hotmail.com,(0904) 694-4591,2024-02-09 13:59:44.733151
1,2,Estiawan,Saragih,estiawansaragih@hotmail.com,+62-62-782-9299,2024-02-09 13:59:44.733151
2,3,Mahesa,Wastuti,mahesawastuti@yahoo.com,+62 (46) 932-1963,2024-02-09 13:59:44.733151
3,4,Drajat,Nainggolan,drajatnainggolan@yahoo.com,+62-042-726-1498,2024-02-09 13:59:44.733151
4,5,Simon,Wacana,simonwacana@hotmail.com,(0007) 948 5515,2024-02-09 13:59:44.733151
...,...,...,...,...,...,...
995,996,Jono,Maryadi,jonomaryadi@hotmail.com,+62 (042) 227-3166,2024-02-09 13:59:44.733151
996,997,Wulan,Novitasari,wulannovitasari@yahoo.com,(0986) 708 8657,2024-02-09 13:59:44.733151
997,998,Balamantri,Salahudin,balamantrisalahudin@yahoo.com,(020) 504 7283,2024-02-09 13:59:44.733151
998,999,Ida,Prayoga,idaprayoga@hotmail.com,0886408953,2024-02-09 13:59:44.733151


### Join

In [33]:
df_cusXres = pd.merge(df_customer, df_reservation, how='inner', on='customer_id')
df_cusXres

,customer_id,first_name,last_name,email,phone,created_at_x,reservation_id,reservation_date,start_date,end_date,total_price,review,rating,created_at_y
0,2,Estiawan,Saragih,estiawansaragih@hotmail.com,+62-62-782-9299,2024-02-09 13:59:44.733151,139,2016-08-30 18:52:49.985,2016-09-20 18:18:02.814,2016-09-25 21:35:42.582,1000000.0,"probably ca n't wrong, hotel welcome respite c...",3,2025-01-26 06:11:04.638130
1,4,Drajat,Nainggolan,drajatnainggolan@yahoo.com,+62-042-726-1498,2024-02-09 13:59:44.733151,49,2021-09-11 04:17:09.909,2021-10-02 02:48:59.764,2021-10-04 09:03:50.491,13000000.0,"fun place went boyfriend time, separated soon ...",2,2025-01-26 06:11:04.616223
2,5,Simon,Wacana,simonwacana@hotmail.com,(0007) 948 5515,2024-02-09 13:59:44.733151,372,2018-05-01 20:26:20.625,2018-05-28 17:52:10.698,2018-06-04 10:00:05.209,9000000.0,excellent hotel money picked hotel based revie...,5,2025-01-26 06:11:04.693162
3,9,Catur,Maryadi,caturmaryadi@yahoo.com,+62 (618) 214 3988,2024-02-09 13:59:44.733151,384,2023-07-04 00:32:43.303,2023-08-03 04:00:49.376,2023-08-15 02:53:20.540,3000000.0,hell argonaut changed planned birthday getaway...,1,2025-01-26 06:11:04.695094
4,11,Najwa,Simbolon,najwasimbolon@yahoo.com,(0276) 921-7073,2024-02-09 13:59:44.733151,164,2018-02-18 05:46:32.044,2018-03-10 12:02:59.063,2018-03-23 09:23:56.599,1000000.0,fantastic place stay sf stayed nov 9 2008 room...,1,2025-01-26 06:11:04.642593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,991,Kasiran,Pranowo,kasiranpranowo@hotmail.com,+62-151-668-9685,2024-02-09 13:59:44.733151,322,2020-05-31 01:48:18.964,2020-06-27 06:31:51.097,2020-07-08 03:27:40.849,13000000.0,great location great service stayed excellent ...,2,2025-01-26 06:11:04.681145
396,993,Harjaya,Hartati,harjayahartati@yahoo.com,+62 (84) 043-5736,2024-02-09 13:59:44.733151,180,2018-05-20 22:30:51.744,2018-06-05 11:57:28.949,2018-06-12 04:43:46.331,3000000.0,lucky discovery having spent months florence y...,2,2025-01-26 06:11:04.647813
397,995,Irfan,Maryadi,irfanmaryadi@gmail.com,+62-057-742-8019,2024-02-09 13:59:44.733151,86,2022-06-25 18:25:26.577,2022-06-27 17:39:58.972,2022-07-02 20:36:45.339,11000000.0,great room location service price recently sta...,3,2025-01-26 06:11:04.624541
398,996,Jono,Maryadi,jonomaryadi@hotmail.com,+62 (042) 227-3166,2024-02-09 13:59:44.733151,241,2016-01-19 10:19:12.907,2016-01-26 08:08:00.170,2016-02-08 21:50:36.990,7000000.0,"askanischer hof, hotel, family stayed askanisc...",1,2025-01-26 06:11:04.663612


In [62]:
final_df = pd.merge(df_cusXres, df, how='inner', on='reservation_id')

In [63]:
final_df.head()

,customer_id,first_name,last_name,email,phone,created_at_x,reservation_id,reservation_date,start_date,end_date,total_price,review,rating,created_at_y,payment_id,provider,account_number,payment_status,payment_date,expire_date
0,2,Estiawan,Saragih,estiawansaragih@hotmail.com,+62-62-782-9299,2024-02-09 13:59:44.733151,139,2016-08-30 18:52:49.985,2016-09-20 18:18:02.814,2016-09-25 21:35:42.582,1000000.0,"probably ca n't wrong, hotel welcome respite c...",3,2025-01-26 06:11:04.638130,139,Ovo,16640674,Success,2016-09-15 14:06:21.733799,2016-09-20 18:18:02.814967
1,4,Drajat,Nainggolan,drajatnainggolan@yahoo.com,+62-042-726-1498,2024-02-09 13:59:44.733151,49,2021-09-11 04:17:09.909,2021-10-02 02:48:59.764,2021-10-04 09:03:50.491,13000000.0,"fun place went boyfriend time, separated soon ...",2,2025-01-26 06:11:04.616223,49,Dana,76380741,Success,2021-09-30 22:51:39.392363,2021-10-02 02:48:59.764712
2,5,Simon,Wacana,simonwacana@hotmail.com,(0007) 948 5515,2024-02-09 13:59:44.733151,372,2018-05-01 20:26:20.625,2018-05-28 17:52:10.698,2018-06-04 10:00:05.209,9000000.0,excellent hotel money picked hotel based revie...,5,2025-01-26 06:11:04.693162,372,Ovo,71579340,Waiting,None,2018-05-28 17:52:10.698413
3,9,Catur,Maryadi,caturmaryadi@yahoo.com,+62 (618) 214 3988,2024-02-09 13:59:44.733151,384,2023-07-04 00:32:43.303,2023-08-03 04:00:49.376,2023-08-15 02:53:20.540,3000000.0,hell argonaut changed planned birthday getaway...,1,2025-01-26 06:11:04.695094,384,Permata,126704617,Failed,None,2023-08-03 04:00:49.376104
4,11,Najwa,Simbolon,najwasimbolon@yahoo.com,(0276) 921-7073,2024-02-09 13:59:44.733151,164,2018-02-18 05:46:32.044,2018-03-10 12:02:59.063,2018-03-23 09:23:56.599,1000000.0,fantastic place stay sf stayed nov 9 2008 room...,1,2025-01-26 06:11:04.642593,164,BSI,124214471,Success,2018-03-07 00:43:44.620445,2018-03-10 12:02:59.063171


### Create New Columns

In [64]:
"""
 column nama full customer
 column total payment dengan IDR
 column email_domain
"""
def handling_col():
    if 'full_name' not in final_df.columns:
        final_df['full_name'] = final_df['first_name'] + ' ' + final_df['last_name']
    if 'total_price' in final_df.columns:
        final_df['currency'] = 'IDR'
    if 'email_domain' not in final_df.columns:
        final_df['email_domain'] = final_df['email'].str.split('@').str[1]
        
    



In [65]:
handling_col()


In [66]:
final_df.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
customer_id,2,4,5,9,11,13,14,14,14,25,...,975,979,986,987,988,991,993,995,996,1000
first_name,Estiawan,Drajat,Simon,Catur,Najwa,Oni,Daru,Daru,Daru,Setya,...,Kanda,Shakila,Febi,Warsita,Gangsa,Kasiran,Harjaya,Irfan,Jono,Heryanto
last_name,Saragih,Nainggolan,Wacana,Maryadi,Simbolon,Mandala,Latupono,Latupono,Latupono,Rajasa,...,Kurniawan,Usamah,Marpaung,Simanjuntak,Suryatmi,Pranowo,Hartati,Maryadi,Maryadi,Siregar
email,estiawansaragih@hotmail.com,drajatnainggolan@yahoo.com,simonwacana@hotmail.com,caturmaryadi@yahoo.com,najwasimbolon@yahoo.com,onimandala@gmail.com,darulatupono@hotmail.com,darulatupono@hotmail.com,darulatupono@hotmail.com,setyarajasa@yahoo.com,...,kandakurniawan@hotmail.com,shakilausamah@gmail.com,febimarpaung@hotmail.com,warsitasimanjuntak@gmail.com,gangsasuryatmi@hotmail.com,kasiranpranowo@hotmail.com,harjayahartati@yahoo.com,irfanmaryadi@gmail.com,jonomaryadi@hotmail.com,heryantosiregar@hotmail.com
phone,+62-62-782-9299,+62-042-726-1498,(0007) 948 5515,+62 (618) 214 3988,(0276) 921-7073,+62-492-871-5943,+62 (534) 599-5145,+62 (534) 599-5145,+62 (534) 599-5145,+62 (037) 621-7629,...,086 554 1872,(076) 001 0041,+62 (029) 365-3098,+62-005-639-8178,(0917) 672-9281,+62-151-668-9685,+62 (84) 043-5736,+62-057-742-8019,+62 (042) 227-3166,0800101666
created_at_x,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,...,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151,2024-02-09 13:59:44.733151
reservation_id,139,49,372,384,164,350,28,172,250,178,...,72,102,64,42,307,322,180,86,241,62
reservation_date,2016-08-30 18:52:49.985000,2021-09-11 04:17:09.909000,2018-05-01 20:26:20.625000,2023-07-04 00:32:43.303000,2018-02-18 05:46:32.044000,2017-05-28 19:21:10.915000,2017-02-04 01:48:49.112000,2018-09-27 01:41:21.886000,2015-04-03 07:34:48.228000,2022-11-24 23:22:42.395000,...,2016-02-03 14:22:04.975000,2018-10-20 20:38:00.445000,2015-11-13 17:41:50.517000,2022-07-31 18:19:08.357000,2023-12-15 09:15:04.802000,2020-05-31 01:48:18.964000,2018-05-20 22:30:51.744000,2022-06-25 18:25:26.577000,2016-01-19 10:19:12.907000,2016-06-22 16:38:50.950000
start_date,2016-09-20 18:18:02.814000,2021-10-02 02:48:59.764000,2018-05-28 17:52:10.698000,2023-08-03 04:00:49.376000,2018-03-10 12:02:59.063000,2017-06-14 00:26:18.932000,2017-02-18 06:59:20.424000,2018-10-18 04:59:54.349000,2015-04-25 09:52:54.235000,2022-11-30 14:20:27.780000,...,2016-02-11 15:01:20.193000,2018-11-16 18:14:40.984000,2015-11-20 06:28:29.081000,2022-08-24 09:26:35.508000,2023-12-26 00:14:26.880000,2020-06-27 06:31:51.097000,2018-06-05 11:57:28.949000,2022-06-27 17:39:58.972000,2016-01-26 08:08:00.170000,2016-07-01 17:54:37.944000
end_date,2016-09-25 21:35:42.582000,2021-10-04 09:03:50.491000,2018-06-04 10:00:05.209000,2023-08-15 02:53:20.540000,2018-03-23 09:23:56.599000,2017-06-18 03:54:42.063000,2017-03-01 01:22:47.384000,2018-10-26 17:53:04.600000,2015-05-07 16:23:54.526000,2022-12-12 10:42:05.923000,...,2016-02-19 12:27:01.926000,2018-11-19 15:06:00.984000,2015-11-25 13:51:50.784000,2022-08-30 18:06:49.685000,2024-01-04 17:40:05.060000,2020-07-08 03:27:40.849000,2018-06-12 04:43:46.331000,2022-07-02 20:36:45.339000,2016-02-08 21:50:36.990000,2016-07-08 15:56:10.137000


### Check Validation

In [67]:
def check_datatype():
    for col in final_df.columns:
        print("====================================")
        print(f"{col} : {final_df[col].dtype}")

In [68]:
check_datatype()

customer_id : int64
first_name : object
last_name : object
email : object
phone : object
created_at_x : datetime64[ns]
reservation_id : int64
reservation_date : datetime64[ns]
start_date : datetime64[ns]
end_date : datetime64[ns]
total_price : float64
review : object
rating : int64
created_at_y : datetime64[ns]
payment_id : int64
provider : object
account_number : int64
payment_status : object
payment_date : object
expire_date : object
full_name : object
currency : object
email_domain : object


In [71]:
def select_col(df):
    # Select specific columns from final_df
    selected_columns = ['reservation_id', 'full_name', 'email', 'email_domain', 'reservation_date', 'payment_date', 'start_date', 'end_date', 'total_price', 'currency', 'provider', 'payment_status']
    df = df[selected_columns]
    return df



In [72]:
final_df = select_col(final_df)
final_df.head()

,reservation_id,full_name,email,email_domain,reservation_date,payment_date,start_date,end_date,total_price,currency,provider,payment_status
0,139,Estiawan Saragih,estiawansaragih@hotmail.com,hotmail.com,2016-08-30 18:52:49.985,2016-09-15 14:06:21.733799,2016-09-20 18:18:02.814,2016-09-25 21:35:42.582,1000000.0,IDR,Ovo,Success
1,49,Drajat Nainggolan,drajatnainggolan@yahoo.com,yahoo.com,2021-09-11 04:17:09.909,2021-09-30 22:51:39.392363,2021-10-02 02:48:59.764,2021-10-04 09:03:50.491,13000000.0,IDR,Dana,Success
2,372,Simon Wacana,simonwacana@hotmail.com,hotmail.com,2018-05-01 20:26:20.625,None,2018-05-28 17:52:10.698,2018-06-04 10:00:05.209,9000000.0,IDR,Ovo,Waiting
3,384,Catur Maryadi,caturmaryadi@yahoo.com,yahoo.com,2023-07-04 00:32:43.303,None,2023-08-03 04:00:49.376,2023-08-15 02:53:20.540,3000000.0,IDR,Permata,Failed
4,164,Najwa Simbolon,najwasimbolon@yahoo.com,yahoo.com,2018-02-18 05:46:32.044,2018-03-07 00:43:44.620445,2018-03-10 12:02:59.063,2018-03-23 09:23:56.599,1000000.0,IDR,BSI,Success


### Check Missing 

In [82]:
def check_missing_value():
    """
    Check for missing values in the DataFrame and calculate the percentage of missing values for each column.

    Parameters:
        final_df (pd.DataFrame): The DataFrame to check for missing values.

    Returns:
        pd.DataFrame: A DataFrame containing the count and percentage of missing values for each column.
    """
    # Check for missing values
    missing_values = final_df.isnull().sum()
    missing_percentage = (missing_values / len(final_df)) * 100

    # Combine results into a DataFrame
    missing_info = pd.DataFrame({
        'Missing Values': missing_values,
        'Missing Percentage': missing_percentage
    })

    return missing_info

In [83]:
check_missing_value()

,Missing Values,Missing Percentage
reservation_id,0,0.0
full_name,0,0.0
email,0,0.0
email_domain,0,0.0
reservation_date,0,0.0
payment_date,84,21.0
start_date,0,0.0
end_date,0,0.0
total_price,0,0.0
currency,0,0.0


In [ ]:
import matplotlib.pyplot as plt

def visualize_missing_values(final_df):
    """
    Visualize the percentage of missing values in each column using a bar plot.
    """
    missing_info = check_missing_value(final_df)
    missing_info['Missing Percentage'].plot(kind='bar', title='Percentage of Missing Values by Column')
    plt.xlabel('Columns')
    plt.ylabel('Missing Percentage')
    plt.show()

# Visualize missing values
visualize_missing_values(final_df)

In [89]:
# Filling Missing Values

def fill_missing_values():
    # Fill missing values in the 'payment_status' column with 'Not Paid'
    final_df['payment_date'].fillna('Missing Data', inplace=True)


    # Check for missing values after filling
    return check_missing_value()

In [90]:
fill_missing_values()

C:\Users\US3R\AppData\Local\Temp\ipykernel_31112\2352423217.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['payment_date'].fillna('Missing Data', inplace=True)
C:\Users\US3R\AppData\Local\Temp\ipykernel_31112\2352423217.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['payment_date'].fillna('Missing Data', inplace=Tru

,Missing Values,Missing Percentage
reservation_id,0,0.0
full_name,0,0.0
email,0,0.0
email_domain,0,0.0
reservation_date,0,0.0
payment_date,0,0.0
start_date,0,0.0
end_date,0,0.0
total_price,0,0.0
currency,0,0.0


# Load

## Create Load Engine

new engine to load

In [98]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

def load_data_to_neon(df, table_name):
    """
    Load data from a DataFrame into a Neon PostgreSQL table.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data to insert.
        table_name (str): The name of the table in the Neon database.
    """
    # Load environment variables from .env file
    load_dotenv()

    # Fetch environment variables
    db_user = os.getenv('DB_USER_Neon')
    db_password = os.getenv('DB_PASSWORD_Neon')
    db_host = os.getenv('DB_HOST_Neon')
    db_name = os.getenv('DB_NAME_Neon')
    db_sslmode = os.getenv('DB_SSLMODE_Neon')

    # Create the connection string
    connection_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}?sslmode={db_sslmode}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    try:
        # Insert DataFrame into Neon
        df.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Data inserted into table '{table_name}' successfully.")
    except Exception as e:
        print(f"Error inserting data into Neon: {e}")

#


In [99]:
load_data_to_neon(df, 'transformed_table')

Data inserted into table 'transformed_table' successfully.


## Retrieve Data from Neon

In [100]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

def retrieve_data_from_neon(query):
    """
    Retrieve data from a Neon PostgreSQL database.

    Parameters:
        query (str): The SQL query to execute.

    Returns:
        pd.DataFrame: A DataFrame containing the query results.
    """
    # Load environment variables from .env file
    load_dotenv()

    # Fetch environment variables
    db_user = os.getenv('DB_USER_Neon')
    db_password = os.getenv('DB_PASSWORD_Neon')
    db_host = os.getenv('DB_HOST_Neon')
    db_name = os.getenv('DB_NAME_Neon')
    db_sslmode = os.getenv('DB_SSLMODE_Neon')

    # Create the connection string
    connection_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}?sslmode={db_sslmode}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    try:
        # Execute the query and load results into a DataFrame
        df = pd.read_sql(query, engine)
        print("Data retrieved successfully!")
        return df
    except Exception as e:
        print(f"Error retrieving data from Neon: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Example SQL query
    query = "SELECT * FROM transformed_table;"  # Replace with your table name

    # Call the function to retrieve data
    data = retrieve_data_from_neon(query)

    # Display the retrieved data
    if data is not None:
        print(data)

Data retrieved successfully!
     payment_id  reservation_id provider  account_number payment_status  \
0             1               1      Ovo        38137149        Success   
1             2               2      BCA        42103729        Success   
2             3               3  Permata        58689635        Success   
3             4               4      BNI       107161965        Success   
4             5               5      BSI        14334131         Failed   
..          ...             ...      ...             ...            ...   
396         397             397  Permata        87836019        Success   
397         398             398  Mandiri        50997930         Failed   
398         399             399      Ovo        44305033        Success   
399         400             400      BNI        84875963        Success   
400         401             401      BCA        11111111        Success   

                   payment_date                 expire_date  
0    202